In [134]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [135]:
X_PATH = "./data/Train.csv"
Y_PATH = "./data/y.csv"

X = pd.read_csv(X_PATH)
y = pd.read_csv(Y_PATH).values.ravel()

In [136]:
X.head()

,ALP_first,ALP_last,ALT_first,ALT_last,AST_first,AST_last,Age,Albumin_first,Albumin_last,BUN_first,...,TroponinT_first,TroponinT_last,UrineOutputSum,WBC_first,WBC_last,Weight,Weight_first,Weight_last,pH_first,pH_last
0,NaN,NaN,NaN,NaN,NaN,NaN,54.0,NaN,NaN,13.0,...,NaN,NaN,NaN,11.2,9.4,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,NaN,16.0,...,NaN,NaN,5.0,7.4,13.3,76.0,80.6,81.6,7.45,7.37
2,127.0,105.0,91.0,75.0,235.0,164.0,44.0,2.7,2.3,8.0,...,NaN,NaN,14.0,4.2,6.2,56.7,56.7,56.7,7.51,7.47
3,105.0,105.0,12.0,12.0,15.0,15.0,68.0,4.4,4.4,23.0,...,NaN,NaN,NaN,11.5,7.9,84.6,84.6,84.6,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,88.0,3.3,3.3,45.0,...,NaN,NaN,NaN,3.8,4.8,NaN,NaN,NaN,NaN,NaN


### Dropping Nan values

In [138]:
drop_threshold = 0.5

count = X.isna().sum()
cols = X.columns[count / len(X) > drop_threshold] #drop columns if nan ratio > threshold
X = X.drop(columns = cols)

### Method1: Median

In [139]:
X = X.fillna(X.median())
X.head()

,Age,BUN_first,BUN_last,CCU,CSRU,Creatinine_first,Creatinine_last,DiasABP_first,DiasABP_highest,DiasABP_last,...,Temp_lowest,Temp_median,UrineOutputSum,WBC_first,WBC_last,Weight,Weight_first,Weight_last,pH_first,pH_last
0,54.0,13.0,8.0,0,0,0.8,0.7,61.0,81.0,60.0,...,35.1,37.70,13.0,11.2,9.4,78.7,81.15,82.2,7.38,7.40
1,76.0,16.0,21.0,0,1,0.8,1.3,67.0,81.0,51.0,...,34.5,37.45,5.0,7.4,13.3,76.0,80.60,81.6,7.45,7.37
2,44.0,8.0,3.0,0,0,0.4,0.3,81.0,81.0,70.0,...,36.7,37.85,14.0,4.2,6.2,56.7,56.70,56.7,7.51,7.47
3,68.0,23.0,10.0,0,0,0.9,0.7,61.0,81.0,60.0,...,35.1,36.40,13.0,11.5,7.9,84.6,84.60,84.6,7.38,7.40
4,88.0,45.0,25.0,0,0,1.0,1.0,61.0,81.0,60.0,...,35.8,37.00,13.0,3.8,4.8,78.7,81.15,82.2,7.38,7.40


In [140]:
X = X.values

### Method2: KNNImpute

In [101]:
imputer = KNNImputer()
X = imputer.fit_transform(X)


### Fitting to a model

In [141]:
model = LogisticRegression(penalty = "l2", max_iter = 500)
scaler = StandardScaler()
kf = KFold(n_splits=20, shuffle=True, random_state=42)

scores = np.zeros(kf.get_n_splits(X))

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, y_train= X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
     
    model.fit(scaler.fit_transform(X_train), y_train)
    scores[i] = model.score(scaler.transform(X_test), y_test)
    
np.mean(scores)

0.8705